In [111]:
import hashlib
import os
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler
import hashlib

In [112]:
n = 256
q = 8380417
k = 3
l = 2 
sigma = 3 

Gq.<z>  = PolynomialRing(GF(q))
Rq.<z> = Gq.quotient(z^n+1)
R.<t> = QQ[]

In [113]:
def gen():
    A = [Rq.random_element() for _ in range(k)]    
    s = [DiscreteGaussianDistributionPolynomialSampler(Rq, 64, sigma)() for _ in range(k)]
    e = [DiscreteGaussianDistributionPolynomialSampler(Rq, 64, sigma)() for _ in range(k)]
    
    t = [A[i] * s[i] + e[i] for i in range(k)]
    
    sk = (s, e, A)
    pk = (A, t)
    
    return (sk, pk)


def sign(m, sk):
    (s, e, A) = sk
    y = Rq.random_element()
    c = [hash(A[i]*y) for i in range(k)] # + parte do m 
    z = [y + s[i]*c[i] for i in range(k)]
    return (z,c)

def verify(m, pk, sig):
    (z, c) = sig
    (A, t) = pk
    w = [A[i]*z[i]-t[i]*c[i] for i in range(k)]
    return c == [hash(w[i]) for i in range(k)]
        
    

In [114]:
m = bin(1024)
(sk,pk) = gen()
(z,c) = sign(m, sk)
print(verify(m, pk, (z,c)))

False
